In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [13]:
df = pd.read_csv("./dataset/Darknet.csv")

In [14]:
df.head()

,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Label.1
0,10.152.152.11-216.58.220.99-57158-443-6,10.152.152.11,57158,216.58.220.99,443,6,24/07/2015 04:09:48 PM,229,1,1,...,0,0,0,0,0.000000e+00,0.000,0.000000e+00,0.000000e+00,Non-Tor,AUDIO-STREAMING
1,10.152.152.11-216.58.220.99-57159-443-6,10.152.152.11,57159,216.58.220.99,443,6,24/07/2015 04:09:48 PM,407,1,1,...,0,0,0,0,0.000000e+00,0.000,0.000000e+00,0.000000e+00,Non-Tor,AUDIO-STREAMING
2,10.152.152.11-216.58.220.99-57160-443-6,10.152.152.11,57160,216.58.220.99,443,6,24/07/2015 04:09:48 PM,431,1,1,...,0,0,0,0,0.000000e+00,0.000,0.000000e+00,0.000000e+00,Non-Tor,AUDIO-STREAMING
3,10.152.152.11-74.125.136.120-49134-443-6,10.152.152.11,49134,74.125.136.120,443,6,24/07/2015 04:09:48 PM,359,1,1,...,0,0,0,0,0.000000e+00,0.000,0.000000e+00,0.000000e+00,Non-Tor,AUDIO-STREAMING
4,10.152.152.11-173.194.65.127-34697-19305-6,10.152.152.11,34697,173.194.65.127,19305,6,24/07/2015 04:09:45 PM,10778451,591,400,...,0,0,0,0,1.437760e+15,3117718.131,1.437760e+15,1.437760e+15,Non-Tor,AUDIO-STREAMING


In [15]:
df["Label"].unique()

array(['Non-Tor', 'NonVPN', 'Tor', 'VPN'], dtype=object)

### Preprocessing

In [16]:
df["Label.2"] = df["Label"]

In [17]:
df['Label.2'].loc[(df['Label'] == 'Non-Tor') | (df['Label'] == 'NonVPN')] = 'Benign'
df['Label.2'].loc[(df['Label'] == 'Tor') | (df['Label'] == 'VPN')] = 'Darknet'


C:\Users\nihar\AppData\Local\Temp\ipykernel_5140\3839010055.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Label.2'].loc[(df['Label'] == 'Non-Tor') | (df['Label'] == 'NonVPN')] = 'Benign'
C:\Users\nihar\AppData\Local\Temp\ipykernel_5140\3839010055.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Label.2'].loc[(df['Label'] == 'Tor') | (df['Label'] == 'VPN')] = 'Darknet'


In [18]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [19]:
df.fillna(0, inplace = True)

In [20]:
df.isnull().sum()

Flow ID     0
Src IP      0
Src Port    0
Dst IP      0
Dst Port    0
           ..
Idle Max    0
Idle Min    0
Label       0
Label.1     0
Label.2     0
Length: 86, dtype: int64

In [21]:
df1 = df.drop(["Timestamp"],axis = 1)

In [22]:
df1 = df1.loc[:, df1.apply(pd.Series.nunique) != 1]

In [23]:
df1.isnull().sum()

Flow ID     0
Src IP      0
Src Port    0
Dst IP      0
Dst Port    0
           ..
Idle Max    0
Idle Min    0
Label       0
Label.1     0
Label.2     0
Length: 70, dtype: int64

In [24]:
df1.drop(["Flow ID"],axis = 1,inplace = True)
df1.drop(["Src IP"],axis = 1,inplace = True)
df1.drop(["Dst IP"],axis = 1,inplace = True)
df1.drop(["Src Port"],axis = 1,inplace = True)
df1.drop(["Dst Port"],axis = 1,inplace = True)
df1.drop(["Flow Duration"],axis = 1,inplace = True)
df1.drop(["Idle Mean"],axis = 1,inplace = True)
df1.drop(["Idle Std"],axis = 1,inplace = True)
df1.drop(["Idle Max"],axis = 1,inplace = True)
df1.drop(["Idle Min"],axis = 1,inplace = True)

In [25]:
from sklearn.preprocessing import OneHotEncoder

df1["Label.2"] = df1["Label.2"].map({"Benign":0,"Darknet":+1})

In [26]:
df1.head()

,Protocol,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,...,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Label,Label.1,Label.2
0,6,1,1,0,0,0,0,0.000000,0.000000,0,...,0,0,0,1892,1047,0,20,Non-Tor,AUDIO-STREAMING,0
1,6,1,1,0,0,0,0,0.000000,0.000000,0,...,0,0,0,1987,1047,0,20,Non-Tor,AUDIO-STREAMING,0
2,6,1,1,0,0,0,0,0.000000,0.000000,0,...,0,0,0,2049,1047,0,20,Non-Tor,AUDIO-STREAMING,0
3,6,1,1,0,0,0,0,0.000000,0.000000,0,...,0,0,0,2008,1047,0,20,Non-Tor,AUDIO-STREAMING,0
4,6,591,400,64530,6659,131,0,109.187817,22.283313,498,...,0,65,6,1382,2320,581,20,Non-Tor,AUDIO-STREAMING,0


In [27]:
df1.rename(columns = {"Label" : "Type", "Label.1" : "Subtype"}, inplace = True)

In [28]:
df1["Subtype"].loc[df1["Subtype"] == "AUDIO-STREAMING"] = "Audio-Streaming"
df1["Subtype"].loc[df1["Subtype"] == "File-transfer"] = "File-Transfer"
df1["Subtype"].loc[df1["Subtype"] == "Video-streaming"] = "Video-Streaming"

C:\Users\nihar\AppData\Local\Temp\ipykernel_5140\2997331084.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Subtype"].loc[df1["Subtype"] == "AUDIO-STREAMING"] = "Audio-Streaming"
C:\Users\nihar\AppData\Local\Temp\ipykernel_5140\2997331084.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Subtype"].loc[df1["Subtype"] == "File-transfer"] = "File-Transfer"
C:\Users\nihar\AppData\Local\Temp\ipykernel_5140\2997331084.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

In [29]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score as acc_score
from sklearn.metrics import classification_report as class_re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split as t_t_s
from sklearn.preprocessing import OneHotEncoder as ohe
subtypes = ['Audio-Streaming', 'Browsing', 'Chat', 'Email', 'File-Transfer', 'P2P', 'VOIP', 'Video-Streaming']
subtypes_ds = pd.DataFrame(subtypes, columns=['subtypes'])

encoder = ohe()


df1["Subtype"] = df1["Subtype"].astype('category')
df1["Subtype_Encoded"] = df1["Subtype"].cat.codes

In [30]:
ds_encoded = pd.DataFrame(encoder.fit_transform(df1[['Subtype_Encoded']]).toarray())
ds_encoded

,0,1,2,3,4,5,6,7
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
141525,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
141526,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
141527,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
141528,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [31]:
df2 = df1.join(ds_encoded)

In [32]:
subtypes = ['Audio-Streaming', 'Browsing', 'Chat', 'Email', 'File-Transfer', 'P2P', 'VOIP', 'Video-Streaming']
df2.rename(columns = {0 : "Audio-Streaming", 1 : "Browsing", 2 : "Chat", 3 : "Email", 4 : "File-Transfer", 5 : "P2P", 6 : "VOIP", 7 : "Video-Streaming"}, inplace = True)

In [33]:
df3 = df2.drop(["Type", "Subtype", "Subtype_Encoded"], axis = 1)

### 1. Binary classification

In [34]:
from sklearn.model_selection import train_test_split

X = df3.drop(["Label.2"], axis = 1)
y = df3["Label.2"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [36]:
from sklearn.ensemble import RandomForestClassifier as RFClass
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
# ranForModel = RFClass(n_estimators=10, criterion = "entropy")
# ranForModel = LogisticRegression(solver='sag')
ranForModel = DecisionTreeClassifier(criterion="entropy")
ranForModel.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy')

In [37]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report as class_re
from sklearn.metrics import confusion_matrix as c_m

ranForPredict = ranForModel.predict(X_test)

print("Predicted Class:",ranForPredict)
print("Confusion Matrix:\n",c_m(y_test ,ranForPredict))
print("Accuracy:", accuracy_score(y_test ,ranForPredict))
print("F_score:", f1_score(y_test ,ranForPredict))
print("Classification Report:\n",class_re(y_test ,ranForPredict))

Predicted Class: [0 1 0 ... 0 0 0]
Confusion Matrix:
 [[34859   311]
 [  325  6964]]
Accuracy: 0.9850208436373914
F_score: 0.9563306783850591
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99     35170
           1       0.96      0.96      0.96      7289

    accuracy                           0.99     42459
   macro avg       0.97      0.97      0.97     42459
weighted avg       0.99      0.99      0.99     42459



### 2. Multiclass Classification 

In [38]:
df4 = df2.drop(['Audio-Streaming', 'Browsing', 'Chat', 'Email', 'File-Transfer', 'P2P', 'VOIP', 'Video-Streaming', 'Type', 'Subtype'], axis = 1)

In [39]:
from sklearn.model_selection import train_test_split

X1 = df4.drop(["Subtype_Encoded"], axis = 1)
y1 = df4["Subtype_Encoded"]

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.30, random_state=42)

In [40]:
from sklearn.ensemble import RandomForestClassifier as RFClass
ranForModel1 = RFClass(n_estimators=10, criterion = "entropy")
ranForModel1.fit(X1_train, y1_train)

RandomForestClassifier(criterion='entropy', n_estimators=10)

In [41]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report as class_re
from sklearn.metrics import confusion_matrix as c_m

ranForPredict = ranForModel1.predict(X1_test)

print("Predicted Class:",ranForPredict)
# print("Confusion Matrix:\n",c_m(y1_test ,ranForPredict))
print("Accuracy:", accuracy_score(y1_test ,ranForPredict))
# print("F_score:", f1_score(y1_test ,ranForPredict))
print("Classification Report:\n",class_re(y1_test ,ranForPredict))

Predicted Class: [5 3 1 ... 0 4 2]
Accuracy: 0.8544478202501237
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.86      0.85      5363
           1       0.87      0.91      0.89      9894
           2       0.73      0.75      0.74      3450
           3       0.63      0.57      0.60      1862
           4       0.80      0.74      0.77      3382
           5       0.96      0.97      0.97     14496
           6       0.65      0.61      0.63      1100
           7       0.68      0.62      0.65      2912

    accuracy                           0.85     42459
   macro avg       0.77      0.75      0.76     42459
weighted avg       0.85      0.85      0.85     42459

